# load necessary pacakges

Set local code paths

In [ ]:
# the path to the scVI github repo https://github.com/YosefLab/scVI
# sys.path.append('/data/yosef2/users/chenling/scVI/')
sys.path.append('/home/angela/src/scVI_TSP/')

# the path to tabulasapiens github repo https://github.com/czbiohub/tabula-sapiens
# sys.path.append('/data/yosef2/users/chenling/tabula-sapiens/')
sys.path.append('/home/angela/sapiens/tabula-sapiens/')


In [1]:
import sys

from scvi.dataset import AnnDatasetFromAnnData
from utils.annotations_utils import *
import scanpy as sc

# if a model has been trained load pretrained model rather than retraining
retrain=False


/home/angela/anaconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
%matplotlib inline

In [3]:
# data_path = '/data/yosef2/users/chenling/TabulaSapiensData/'
# model_path = '/data/yosef2/users/chenling/TabulaSapiens/scVImodels/Pilot2_nova2/'
# res_path = '/data/yosef2/users/chenling/TabulaSapiens/Pilot2_rerun/'

data_path = '/mnt/ibm_lg/angela/sapiens/data/'
model_path = '/mnt/ibm_lg/angela/sapiens/data/scVI_models/'
res_path = '/mnt/ibm_lg/angela/sapiens/data/'

# if not os.path.exists(model_path):
#     os.mkdir(model_path)
# if not os.path.exists(model_path+'scanvi/'):
#     os.mkdir(model_path+'scanvi/')
# if not os.path.exists(res_path):
#     os.mkdir(res_path)

# Data to use

In [5]:
print("tsp1_10x")
tsp1_10x = sc.read_h5ad(
    "/mnt/ibm_lg/angela/sapiens/TSP1/TSP1-10X-min_genes_200-min_reads_2500.h5ad")
display(tsp1_10x)
display(np.min(tsp1_10x.obs["n_counts"]),np.min(tsp1_10x.obs["n_genes"]))

print("tsp1_ss2")
tsp1_ss2 = sc.read_h5ad(
    "/mnt/ibm_lg/angela/sapiens/TSP1/smartseq2/TSP1_SS2_gencode_filtered_raw.h5ad")
display(tsp1_ss2)
display(np.min(tsp1_ss2.obs["n_counts"]),np.min(tsp1_ss2.obs["n_genes"]))


print("tsp2_10x")
tsp2_10x = sc.read_h5ad(
    "/mnt/ibm_lg/angela/sapiens/TSP2/TSP2-10X-novaseq_rerun-nextseq-min_genes_200-min_reads_2500.h5ad")
display(tsp2_10x)
display(np.min(tsp2_10x.obs["n_counts"]),np.min(tsp2_10x.obs["n_genes"]))

print("tsp2_ss2_1")
tsp2_ss2_1 = sc.read_h5ad(
    "/mnt/ibm_lg/angela/sapiens/TSP2/smartseq2/TSP2_SS2_no_official_metadata_filtered_raw_batch1.h5ad")
display(tsp2_ss2_1)
display(np.min(tsp2_ss2_1.obs["n_counts"]),np.min(tsp2_ss2_1.obs["n_genes"]))

print("tsp2_ss2_2")
tsp2_ss2_2 = sc.read_h5ad(
    "/mnt/ibm_lg/angela/sapiens/TSP2/smartseq2/TSP2_SS2_no_official_metadata_filtered_raw_batch2.h5ad")
display(tsp2_ss2_2)
display(np.min(tsp2_ss2_2.obs["n_counts"]),np.min(tsp2_ss2_2.obs["n_genes"]))



tsp1_10x


AnnData object with n_obs × n_vars = 44199 × 58870 
    obs: '10X_run', 'n_counts', 'n_genes', 'replicate', 'tissue', 'method', 'sequencer', 'pilot', 'sex'
    var: 'gene_ids', 'feature_types'

2500.0

200

tsp1_ss2


...

200

tsp2_ss2_2


AnnData object with n_obs × n_vars = 4494 × 58962 
    obs: 'pilot', 'Tissue', 'AnatomicalPosition', 'method', 'cDNAPlate', 'LibraryPlateID', 'population', 'well_seq', 'cell_id', 'n_genes', 'n_counts'
    var: 'n_cells'

5029.0

200

# Prepare Pilot1 Data

## Filter duplicated barcodes in 10X

In [6]:
display(tsp1_10x.shape)
idx = np.asarray(FindUniqueCells(tsp1_10x))
tsp1_10x = tsp1_10x[idx].copy()
display(tsp1_10x.shape)

(44199, 58870)

(43457, 58870)

In [7]:
sc.pp.filter_cells(tsp1_10x,min_counts=0)
sc.pp.filter_cells(tsp1_10x,min_genes=0)
sc.pp.filter_genes(tsp1_10x,min_cells=0)

display(np.min(tsp1_10x.obs["n_counts"]),np.min(tsp1_10x.obs["n_genes"]))

2500.0

200

## Concatenate both technologies

In [10]:
tsp1_ss2.var

,n_cells
ENSG00000000003.14,581
ENSG00000000005.6,68
ENSG00000000419.12,753
ENSG00000000457.14,233
ENSG00000000460.17,152
...,...
ERCC-00164,4
ERCC-00165,164
ERCC-00168,3
ERCC-00170,39


In [11]:
tsp1_10x.var

,gene_ids,feature_types,n_cells
DDX11L1,ENSG00000223972.5,Gene Expression,0
WASH7P,ENSG00000227232.5,Gene Expression,1
MIR6859-1,ENSG00000278267.1,Gene Expression,0
MIR1302-2HG,ENSG00000243485.5,Gene Expression,0
MIR1302-2,ENSG00000284332.1,Gene Expression,0
...,...,...,...
MT-ND6,ENSG00000198695.2,Gene Expression,19450
MT-TE,ENSG00000210194.1,Gene Expression,169
MT-CYB,ENSG00000198727.2,Gene Expression,43309
MT-TT,ENSG00000210195.2,Gene Expression,2170


In [13]:
tsp1_10x.var['gene_symbol'] = tsp1_10x.var.index
tsp1_10x.var = tsp1_10x.var.set_index('gene_ids')
tsp1_10x.var

,feature_types,n_cells,gene_symbol
gene_ids,,,
ENSG00000223972.5,Gene Expression,0,DDX11L1
ENSG00000227232.5,Gene Expression,1,WASH7P
ENSG00000278267.1,Gene Expression,0,MIR6859-1
ENSG00000243485.5,Gene Expression,0,MIR1302-2HG
ENSG00000284332.1,Gene Expression,0,MIR1302-2
...,...,...,...
ENSG00000198695.2,Gene Expression,19450,MT-ND6
ENSG00000210194.1,Gene Expression,169,MT-TE
ENSG00000198727.2,Gene Expression,43309,MT-CYB


In [14]:
pilot1 = tsp1_10x.concatenate(tsp1_ss2)
pilot1

AnnData object with n_obs × n_vars = 47844 × 58870 
    obs: '10X_run', 'Additional Notes', 'AnatomicalPosition', 'BA nM', 'Barcode', 'Excluded', 'Investigator', 'LibraryPlateID', 'NovaSeqRun', 'PrimarySort', 'SortDirection', 'SortOrder', 'Tissue', 'Well_Range', 'batch', 'bead cleanup', 'cDNA Dilute', 'cDNA Prep', 'cDNAPlate', 'cell_id', 'lib prep date', 'library prep', 'method', 'n_counts', 'n_genes', 'pilot', 'plate', 'replicate', 'sequencer', 'sex', 'tissue', 'well_metadata', 'well_seq'
    var: 'feature_types-0', 'n_cells-0', 'gene_symbol-0', 'n_cells-1'

In [15]:
pilot1.var

,feature_types-0,n_cells-0,gene_symbol-0,n_cells-1
ENSG00000223972.5,Gene Expression,0,DDX11L1,2
ENSG00000227232.5,Gene Expression,1,WASH7P,529
ENSG00000278267.1,Gene Expression,0,MIR6859-1,7
ENSG00000243485.5,Gene Expression,0,MIR1302-2HG,4
ENSG00000284332.1,Gene Expression,0,MIR1302-2,0
...,...,...,...,...
ENSG00000198695.2,Gene Expression,19450,MT-ND6,3662
ENSG00000210194.1,Gene Expression,169,MT-TE,1289
ENSG00000198727.2,Gene Expression,43309,MT-CYB,4070
ENSG00000210195.2,Gene Expression,2170,MT-TT,2


In [33]:
display(np.min(pilot1.obs["n_counts"]),np.min(pilot1.obs["n_genes"]))

2500.0

200

In [32]:
pilot1

AnnData object with n_obs × n_vars = 47772 × 58870 
    obs: '10X_run', 'Additional Notes', 'AnatomicalPosition', 'BA nM', 'Barcode', 'Excluded', 'Investigator', 'LibraryPlateID', 'NovaSeqRun', 'PrimarySort', 'SortDirection', 'SortOrder', 'Tissue', 'Well_Range', 'batch', 'bead cleanup', 'cDNA Dilute', 'cDNA Prep', 'cDNAPlate', 'cell_id', 'lib prep date', 'library prep', 'method', 'n_counts', 'n_genes', 'pilot', 'plate', 'replicate', 'sequencer', 'sex', 'tissue', 'well_metadata', 'well_seq'
    var: 'feature_types-0', 'n_cells-0', 'gene_symbol-0', 'n_cells-1', 'n_cells'

In [31]:
pilot1.obs.columns

Index(['10X_run', 'Additional Notes', 'AnatomicalPosition', 'BA nM', 'Barcode',
       'Excluded', 'Investigator', 'LibraryPlateID', 'NovaSeqRun',
       'PrimarySort', 'SortDirection', 'SortOrder', 'Tissue', 'Well_Range',
       'batch', 'bead cleanup', 'cDNA Dilute', 'cDNA Prep', 'cDNAPlate',
       'cell_id', 'lib prep date', 'library prep', 'method', 'n_counts',
       'n_genes', 'pilot', 'plate', 'replicate', 'sequencer', 'sex', 'tissue',
       'well_metadata', 'well_seq'],
      dtype='object')

In [35]:
pilot1.write_h5ad(data_path + 'Pilot1_filtered_ensembl_id.h5ad')

... storing '10X_run' as categorical
... storing 'Additional Notes' as categorical
... storing 'AnatomicalPosition' as categorical
... storing 'Barcode' as categorical
... storing 'Excluded' as categorical
... storing 'Investigator' as categorical
... storing 'LibraryPlateID' as categorical
... storing 'NovaSeqRun' as categorical
... storing 'PrimarySort' as categorical
... storing 'SortDirection' as categorical
... storing 'SortOrder' as categorical
... storing 'Tissue' as categorical
... storing 'Well_Range' as categorical
... storing 'bead cleanup' as categorical
... storing 'cDNA Dilute' as categorical
... storing 'cDNA Prep' as categorical
... storing 'cDNAPlate' as categorical
... storing 'cell_id' as categorical
... storing 'lib prep date' as categorical
... storing 'library prep' as categorical
... storing 'method' as categorical
... storing 'pilot' as categorical
... storing 'plate' as categorical
... storing 'replicate' as categorical
... storing 'sequencer' as categorical
..

# Prepare Pilot2 Data

In [36]:
display(tsp2_10x)
tsp2_10x_nova = tsp2_10x[tsp2_10x.obs['sequencer']=='novaseq'].copy()
display(tsp2_10x_nova)
idx1 = np.asarray(FindUniqueCells(tsp2_10x_nova))
tsp2_10x_nova = tsp2_10x_nova[idx1].copy()
display(tsp2_10x_nova)

tsp2_10x_next = tsp2_10x[tsp2_10x.obs['sequencer']=='nextseq'].copy()
display(tsp2_10x_next)
idx2 = np.asarray(FindUniqueCells(tsp2_10x_next))
tsp2_10x_next = tsp2_10x_next[idx2].copy()
display(tsp2_10x_next)

tsp2_10x = tsp2_10x_nova.concatenate(tsp2_10x_next).copy()
display(tsp2_10x)

AnnData object with n_obs × n_vars = 154053 × 58870 
    obs: '10X_run', 'method', 'n_counts', 'n_genes', 'notes', 'pilot', 'replicate', 'sample', 'sequencer', 'subtissue', 'tissue', 'sex'
    var: 'gene_ids', 'feature_types'

AnnData object with n_obs × n_vars = 151483 × 58870 
    obs: '10X_run', 'method', 'n_counts', 'n_genes', 'notes', 'pilot', 'replicate', 'sample', 'sequencer', 'subtissue', 'tissue', 'sex'
    var: 'gene_ids', 'feature_types'

AnnData object with n_obs × n_vars = 145691 × 58870 
    obs: '10X_run', 'method', 'n_counts', 'n_genes', 'notes', 'pilot', 'replicate', 'sample', 'sequencer', 'subtissue', 'tissue', 'sex'
    var: 'gene_ids', 'feature_types'

AnnData object with n_obs × n_vars = 2570 × 58870 
    obs: '10X_run', 'method', 'n_counts', 'n_genes', 'notes', 'pilot', 'replicate', 'sample', 'sequencer', 'subtissue', 'tissue', 'sex'
    var: 'gene_ids', 'feature_types'

AnnData object with n_obs × n_vars = 2568 × 58870 
    obs: '10X_run', 'method', 'n_counts', 'n_genes', 'notes', 'pilot', 'replicate', 'sample', 'sequencer', 'subtissue', 'tissue', 'sex'
    var: 'gene_ids', 'feature_types'

AnnData object with n_obs × n_vars = 148259 × 58870 
    obs: '10X_run', 'batch', 'method', 'n_counts', 'n_genes', 'notes', 'pilot', 'replicate', 'sample', 'sequencer', 'sex', 'subtissue', 'tissue'
    var: 'gene_ids', 'feature_types'

In [37]:
tsp2_ss2_1.var

,n_cells
ENSG00000000003.14,359
ENSG00000000005.6,18
ENSG00000000419.12,775
ENSG00000000457.14,254
ENSG00000000460.17,117
...,...
ERCC-00164,1
ERCC-00165,110
ERCC-00168,0
ERCC-00170,19


In [38]:
tsp2_ss2_2.var

,n_cells
ENSG00000000003.14,318
ENSG00000000005.6,35
ENSG00000000419.12,655
ENSG00000000457.14,280
ENSG00000000460.17,168
...,...
ERCC-00164,4
ERCC-00165,155
ERCC-00168,3
ERCC-00170,32


In [41]:
tsp2_10x.var

,gene_ids,feature_types
DDX11L1,ENSG00000223972.5,Gene Expression
WASH7P,ENSG00000227232.5,Gene Expression
MIR6859-1,ENSG00000278267.1,Gene Expression
MIR1302-2HG,ENSG00000243485.5,Gene Expression
MIR1302-2,ENSG00000284332.1,Gene Expression
...,...,...
MT-ND6,ENSG00000198695.2,Gene Expression
MT-TE,ENSG00000210194.1,Gene Expression
MT-CYB,ENSG00000198727.2,Gene Expression
MT-TT,ENSG00000210195.2,Gene Expression


In [42]:
tsp2_10x.var['gene_symbol'] = tsp2_10x.var.index
tsp2_10x.var = tsp2_10x.var.set_index('gene_ids')

tsp2_10x.var

,feature_types,gene_symbol
gene_ids,,
ENSG00000223972.5,Gene Expression,DDX11L1
ENSG00000227232.5,Gene Expression,WASH7P
ENSG00000278267.1,Gene Expression,MIR6859-1
ENSG00000243485.5,Gene Expression,MIR1302-2HG
ENSG00000284332.1,Gene Expression,MIR1302-2
...,...,...
ENSG00000198695.2,Gene Expression,MT-ND6
ENSG00000210194.1,Gene Expression,MT-TE
ENSG00000198727.2,Gene Expression,MT-CYB


In [43]:
pilot2 = tsp2_10x.concatenate(tsp2_ss2_1)
pilot2.obs = pilot2.obs.drop('batch',axis=1)
pilot2 = pilot2.concatenate(tsp2_ss2_2)
pilot2

AnnData object with n_obs × n_vars = 156651 × 58870 
    obs: '10X_run', 'AnatomicalPosition', 'LibraryPlateID', 'Tissue', 'batch', 'cDNAPlate', 'cell_id', 'method', 'n_counts', 'n_genes', 'notes', 'pilot', 'population', 'replicate', 'sample', 'sequencer', 'sex', 'subtissue', 'tissue', 'well_seq'
    var: 'feature_types-0-0', 'gene_symbol-0-0', 'n_cells-1-0', 'n_cells-1'

In [44]:
pilot2.write_h5ad(data_path + 'Pilot2_filtered_ensembl_id.h5ad')

... storing '10X_run' as categorical
... storing 'AnatomicalPosition' as categorical
... storing 'LibraryPlateID' as categorical
... storing 'Tissue' as categorical
... storing 'cDNAPlate' as categorical
... storing 'cell_id' as categorical
... storing 'method' as categorical
... storing 'notes' as categorical
... storing 'population' as categorical
... storing 'replicate' as categorical
... storing 'sample' as categorical
... storing 'sequencer' as categorical
... storing 'sex' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical
... storing 'well_seq' as categorical


# Combine all data

In [ ]:
pilot1 = sc.read_h5ad(data_path + 'Pilot1_filtered_ensembl_id.h5ad')
pilot2 = sc.read_h5ad(data_path + 'Pilot2_filtered_ensembl_id.h5ad')

In [45]:
pilot1.obs['pilot'] = 'TSP1'
pilot2.obs['pilot'] = 'TSP2'

pilot1.obs = pilot1.obs.drop('batch',axis=1)
pilot2.obs = pilot2.obs.drop('batch',axis=1)

adata = pilot1.concatenate(pilot2)
adata

AnnData object with n_obs × n_vars = 204423 × 58870 
    obs: '10X_run', 'Additional Notes', 'AnatomicalPosition', 'BA nM', 'Barcode', 'Excluded', 'Investigator', 'LibraryPlateID', 'NovaSeqRun', 'PrimarySort', 'SortDirection', 'SortOrder', 'Tissue', 'Well_Range', 'batch', 'bead cleanup', 'cDNA Dilute', 'cDNA Prep', 'cDNAPlate', 'cell_id', 'lib prep date', 'library prep', 'method', 'n_counts', 'n_genes', 'notes', 'pilot', 'plate', 'population', 'replicate', 'sample', 'sequencer', 'sex', 'subtissue', 'tissue', 'well_metadata', 'well_seq'
    var: 'feature_types-0-0', 'n_cells-0-0', 'gene_symbol-0-0', 'n_cells-1-0', 'n_cells-0', 'feature_types-0-0-1', 'gene_symbol-0-0-1', 'n_cells-1-0-1', 'n_cells-1-1'

In [46]:
np.min(adata.obs["n_counts"]),np.min(adata.obs["n_genes"])

(2500.0, 200)

## Clean metadata

In [47]:
adata.obs.columns

Index(['10X_run', 'Additional Notes', 'AnatomicalPosition', 'BA nM', 'Barcode',
       'Excluded', 'Investigator', 'LibraryPlateID', 'NovaSeqRun',
       'PrimarySort', 'SortDirection', 'SortOrder', 'Tissue', 'Well_Range',
       'batch', 'bead cleanup', 'cDNA Dilute', 'cDNA Prep', 'cDNAPlate',
       'cell_id', 'lib prep date', 'library prep', 'method', 'n_counts',
       'n_genes', 'notes', 'pilot', 'plate', 'population', 'replicate',
       'sample', 'sequencer', 'sex', 'subtissue', 'tissue', 'well_metadata',
       'well_seq'],
      dtype='object')

In [48]:
adata.obs['sex'] = 'female'

adata.obs['method'].replace('droplet', '10X',inplace=True)
adata.obs['method'].replace('SS2', 'smartseq2',inplace=True)
adata.obs['method'].replace('nan', 'smartseq2',inplace=True)

adata.obs.loc[adata.obs['pilot']=='TSP1','sequencer'] = 'novaseq'
adata.obs.loc[adata.obs['method']=='smartseq2','sequencer'] = 'novaseq'

adata.obs['10X_run'] = adata.obs['10X_run']

adata.obs.loc[adata.obs['pilot']=='TSP1','notes'] = adata.obs.loc[adata.obs['pilot']=='TSP1','Additional Notes']

adata.obs.loc[adata.obs['pilot']=='TSP1','cDNAPlate'] = adata.obs.loc[adata.obs['pilot']=='TSP1','plate']

adata.obs.loc[adata.obs['pilot']=='TSP1','population'] = adata.obs.loc[adata.obs['pilot']=='TSP1','PrimarySort']

adata.obs.loc[adata.obs['method']=='smartseq2','tissue'] = adata.obs.loc[adata.obs['method']=='smartseq2','Tissue']

adata.obs.loc[adata.obs['subtissue'].isna(),'subtissue'] = adata.obs.loc[adata.obs['subtissue'].isna(),'AnatomicalPosition']

adata.obs['10X_barcode'] = [i.split("-")[0] for i in adata.obs.index]
adata.obs.loc[adata.obs['method']=='smartseq2','10X_barcode'] = 'nan'

adata.obs['aux_cell_id'] = adata.obs['10X_barcode'] + "_" + adata.obs['10X_run']
adata.obs.loc[adata.obs['cell_id']=='nan','cell_id'] = adata.obs.loc[adata.obs['cell_id']=='nan','aux_cell_id']


In [49]:
adata.obs.rename(columns = {'replicate':'10X_replicate',
                           'SortOrder':'smartseq2_sort_order',
                            'Barcode':'smartseq2_barcode',
                            'LibraryPlateID':'smartseq2_library_plate', #for TSP1, LibraryPlateID == cDNAPlate+Blue
                            'SortDirection':'smartseq2_sorting_direction',
                            'sample':'10X_sample',
                           'subtissue':'anatomical_position',
                           'cDNAPlate':'smartseq2_cDNA_Plate',
                           'population':'smartseq2_sorted_population',
                           'well_seq':'smartseq2_well_seq',
                           "pilot":"donor"}, 
                 inplace=True)

In [50]:
adata.obs = adata.obs.drop(
    ['BA nM','Excluded', 'Investigator','lib prep date', 'library prep',"NovaSeqRun",
    "bead cleanup","batch","cDNA Dilute",'cDNA Prep','Well_Range', 'Additional Notes','PrimarySort',
    "AnatomicalPosition","Tissue",'well_metadata','aux_cell_id','n_counts', 'n_genes','plate'], 
    axis = 1)

In [51]:
adata.obs.columns

Index(['10X_run', 'smartseq2_barcode', 'smartseq2_library_plate',
       'smartseq2_sorting_direction', 'smartseq2_sort_order',
       'smartseq2_cDNA_Plate', 'cell_id', 'method', 'notes', 'donor',
       'smartseq2_sorted_population', '10X_replicate', '10X_sample',
       'sequencer', 'sex', 'anatomical_position', 'tissue',
       'smartseq2_well_seq', '10X_barcode'],
      dtype='object')

In [52]:
adata.obs.replace('NA','nan',inplace=True)

In [56]:
adata = adata[adata.obs['tissue']!='nan'].copy()
adata.obs['tissue'] = adata.obs['tissue'].str.title()
for t in set(adata.obs['tissue']):
    if t in ['Endopancreas','Pancreasendocrine']:
        adata.obs.loc[adata.obs['tissue'] == t,'anatomical_position'] = 'Endocrine'
        adata.obs.loc[adata.obs['tissue'] == t,'tissue'] = 'Pancreas'
    if t in ['Pancreasexocrine']:
        adata.obs.loc[adata.obs['tissue'] == t,'anatomical_position'] = 'Exocrine'
        adata.obs.loc[adata.obs['tissue'] == t,'tissue'] = 'Pancreas'
    if t in ['Exopancreas1','Exopancreas2']:
        adata.obs.loc[adata.obs['tissue'] == t,'anatomical_position'] = 'Exocrine'
        adata.obs.loc[adata.obs['tissue'] == t,'smartseq2_sort_order'] = np.int(t.split('pancreas')[1])
        adata.obs.loc[adata.obs['tissue'] == t,'tissue'] = 'Pancreas'
    if t in ['Bm']:
        adata.obs.loc[adata.obs['tissue'] == t,'tissue'] = 'Bone_Marrow'
    if t in ['Li']:
        adata.obs.loc[adata.obs['tissue'] == t,'tissue'] = 'Large_Intestine'
    if t in ['Si']:
        adata.obs.loc[adata.obs['tissue'] == t,'tissue']='Small_Intestine'
    if t in ['Lymphnode']:
        adata.obs.loc[adata.obs['tissue'] == t,'tissue'] = 'Lymph_Node'
    if t in ['Lungneuron']:
        adata.obs.loc[adata.obs['tissue'] == t,'anatomical_position'] = 'Neuron'
        adata.obs.loc[adata.obs['tissue'] == t,'tissue'] = 'Lung'

In [57]:
adata.obs.columns

Index(['10X_run', 'smartseq2_barcode', 'smartseq2_library_plate',
       'smartseq2_sorting_direction', 'smartseq2_sort_order',
       'smartseq2_cDNA_Plate', 'cell_id', 'method', 'notes', 'donor',
       'smartseq2_sorted_population', '10X_replicate', '10X_sample',
       'sequencer', 'sex', 'anatomical_position', 'tissue',
       'smartseq2_well_seq', '10X_barcode'],
      dtype='object')

In [58]:
pd.DataFrame(
    adata.obs.groupby(['tissue']).count())

,10X_run,smartseq2_barcode,smartseq2_library_plate,smartseq2_sorting_direction,smartseq2_sort_order,smartseq2_cDNA_Plate,cell_id,method,notes,donor,smartseq2_sorted_population,10X_replicate,10X_sample,sequencer,sex,anatomical_position,smartseq2_well_seq,10X_barcode
tissue,,,,,,,,,,,,,,,,,,
Bladder,25768,15088,25768,15088,15088,25768,25768,25768,25768,25768,25768,25768,10680,25768,25768,25768,25768,25768
Blood,16064,4192,16064,4192,4192,16064,16064,16064,16064,16064,16064,16064,11872,16064,16064,16064,16064,16064
Bone_Marrow,3190,0,3190,0,0,3190,3190,3190,3190,3190,3190,3190,3190,3190,3190,3190,3190,3190
Fat,301,0,301,0,0,301,301,301,301,301,301,301,301,301,301,301,301,301
Heart,269,0,269,0,0,269,269,269,269,269,269,269,269,269,269,269,269,269
Kidney,24286,0,24286,0,0,24286,24286,24286,24286,24286,24286,24286,24286,24286,24286,24286,24286,24286
Large_Intestine,8213,0,8213,0,0,8213,8213,8213,8213,8213,8213,8213,8213,8213,8213,8213,8213,8213
Lung,37175,15512,37175,15512,15512,37175,37175,37175,37175,37175,37175,37175,21663,37175,37175,37175,37175,37175
Lymph_Node,10075,0,10075,0,0,10075,10075,10075,10075,10075,10075,10075,10075,10075,10075,10075,10075,10075


In [60]:
pd.DataFrame(
    adata.obs.groupby(['sex','donor','tissue','anatomical_position','method',
                   '10X_run','10X_barcode',
                       'smartseq2_barcode',
                       'smartseq2_sorting_direction',
                       'smartseq2_sort_order',
                   'smartseq2_cDNA_Plate',
                   'smartseq2_library_plate',
                   'smartseq2_sorted_population',
                   'smartseq2_well_seq',
                   'sequencer','notes']
                 )['cell_id'].count()
)

cell_id
sex    donor tissue   anatomical_position method    10X_run        10X_barcode      smartseq2_barcode smartseq2_sorting_direction smartseq2_sort_order smartseq2_cDNA_Plate smartseq2_library_plate smartseq2_sorted_population smartseq2_well_seq sequencer notes                           
female TSP1  Bladder  nan                 10X       TSP1_bladder_1 AAACCCAGTGTAGCAG nan               nan                         nan                  nan                  nan                     nan                         nan                novaseq   nan                            1
                                                                   AAACCCATCAAGTCTG nan               nan                         nan                  nan                  nan                     nan                         nan                novaseq   nan                            1
                                                                   AAACCCATCCTTCGAC nan               nan                         nan                  nan                  nan                     nan                         nan                novaseq   nan                            1
                                                                   AAACCCATCGAAGGAC nan               nan                         nan                  nan                  nan                     nan                         nan                novaseq   nan                            1
                                                                   AAACGAAAGCTCCGAC nan               nan                         nan                  nan                  nan                     nan                         nan                novaseq   nan                            1
...                                                                                                                                                                                                                                                                                       ...
             Pancreas Exocrine            smartseq2 nan            nan              BC4               Row                         Plate 1              B107816              B107816Blue             Epithelial                  P11                novaseq   Short Dissociation Time        1
                                                                                                                                                                                                                                P19                novaseq   Short Dissociation Time        1
                                                                                                                                                                                                                                P2                 novaseq   Short Dissociation Time        1
                                                                                                                                                                                                                                P3                 novaseq   Short Dissociation Time        1
                                                                                                                                                                                                                                P9                 novaseq   Short Dissociation Time        1

[47772 rows x 1 columns]

In [59]:
pd.DataFrame(
    adata.obs.groupby(['tissue','anatomical_position']).count())

10X_run  smartseq2_barcode  \
tissue          anatomical_position                               
Bladder         nan                    25768              15088   
Blood           nan                    16064               4192   
Bone_Marrow     nan                      329                  0   
                vertebralbody           2861                  0   
Fat             MAT                      115                  0   
                SCAT                     186                  0   
Heart           ventricle                269                  0   
Kidney          nan                    24286                  0   
Large_Intestine distal                  3990                  0   
                nan                      394                  0   
                proximal                3829                  0   
Lung            MedialDistal             332                332   
                Neuron                   298                  0   
                Proximal                 398                398   
                nan                    15268              14782   
                proxmedialdistal       20879                  0   
Lymph_Node      nan                    10075                  0   
Muscle          diaphragm               5075                  0   
                nan                     5366               4051   
                rectusabdominus        11335                  0   
Pancreas        Endocrine               2471               2471   
                Exocrine                6458               6458   
Skin            abdomen                    2                  0   
                nan                      870                  0   
Small_Intestine distal                   823                  0   
                nan                      633                  0   
                proximal                8123                  0   
Spleen          nan                     8389                  0   
Thymus          nan                    12914                  0   
Trachea         nan                     8039                  0   
Vasculature     Aorta                   8311                  0   
                nan                      273                  0   

                                     smartseq2_library_plate  \
tissue          anatomical_position                            
Bladder         nan                                    25768   
Blood           nan                                    16064   
Bone_Marrow     nan                                      329   
                vertebralbody                           2861   
Fat             MAT                                      115   
                SCAT                                     186   
Heart           ventricle                                269   
Kidney          nan                                    24286   
Large_Intestine distal                                  3990   
                nan                                      394   
                proximal                                3829   
Lung            MedialDistal                             332   
                Neuron                                   298   
                Proximal                                 398   
                nan                                    15268   
                proxmedialdistal                       20879   
Lymph_Node      nan                                    10075   
Muscle          diaphragm                               5075   
                nan                                     5366   
                rectusabdominus                        11335   
Pancreas        Endocrine                               2471   
                Exocrine                                6458   
Skin            abdomen                                    2   
                nan                                      870   
Small_Intestine distal                                   823   
                nan           

## Refilter data after removal of ERCCs

Add `n_genes`, `n_counts` and `n_cells` to the combined object

In [61]:
sc.pp.filter_cells(adata,min_counts=0)
sc.pp.filter_cells(adata,min_genes=0)
sc.pp.filter_genes(adata,min_cells=0)

adata

AnnData object with n_obs × n_vars = 204423 × 58870 
    obs: '10X_run', 'smartseq2_barcode', 'smartseq2_library_plate', 'smartseq2_sorting_direction', 'smartseq2_sort_order', 'smartseq2_cDNA_Plate', 'cell_id', 'method', 'notes', 'donor', 'smartseq2_sorted_population', '10X_replicate', '10X_sample', 'sequencer', 'sex', 'anatomical_position', 'tissue', 'smartseq2_well_seq', '10X_barcode', 'n_counts', 'n_genes'
    var: 'feature_types-0-0', 'n_cells-0-0', 'gene_symbol-0-0', 'n_cells-1-0', 'n_cells-0', 'feature_types-0-0-1', 'gene_symbol-0-0-1', 'n_cells-1-0-1', 'n_cells-1-1', 'n_cells'

In [62]:
np.min(adata.obs["n_counts"]),np.min(adata.obs["n_genes"])

(288.0, 187)

Check for `n_genes` and `n_counts` after concatenation

In [63]:
adata_smartseq2 = adata.obs[adata.obs['method']=='smartseq2'].copy()
adata_10X = adata.obs[adata.obs['method']=='10X'].copy()

Re-filter smartseq2

In [65]:
adata_smartseq2 = adata[adata.obs['method']=='smartseq2'].copy()
display(adata_smartseq2.shape)
adata_smartseq2 = adata_smartseq2[adata_smartseq2.obs['n_genes']>200]
display(adata_smartseq2.shape)
adata_smartseq2 = adata_smartseq2[adata_smartseq2.obs['n_counts']>5000]
display(adata_smartseq2.shape)

(12707, 58870)

(12673, 58870)

(12497, 58870)

Re-filter 10X

In [66]:
adata_10X = adata[adata.obs['method']=='10X'].copy()
display(adata_10X.shape)
adata_10X = adata_10X[adata_10X.obs['n_genes']>200]
display(adata_10X.shape)
adata_10X = adata_10X[adata_10X.obs['n_counts']>2500]
display(adata_10X.shape)

(191716, 58870)

(191624, 58870)

(191585, 58870)

In [67]:
adata_idx = list(adata_smartseq2.obs.index) + list(adata_10X.obs.index)
len(adata_idx)

204082

In [68]:
adata = adata[adata.obs.index.isin(adata_idx),:].copy()

## Write combined object

In [69]:
adata

AnnData object with n_obs × n_vars = 204082 × 58870 
    obs: '10X_run', 'smartseq2_barcode', 'smartseq2_library_plate', 'smartseq2_sorting_direction', 'smartseq2_sort_order', 'smartseq2_cDNA_Plate', 'cell_id', 'method', 'notes', 'donor', 'smartseq2_sorted_population', '10X_replicate', '10X_sample', 'sequencer', 'sex', 'anatomical_position', 'tissue', 'smartseq2_well_seq', '10X_barcode', 'n_counts', 'n_genes'
    var: 'feature_types-0-0', 'n_cells-0-0', 'gene_symbol-0-0', 'n_cells-1-0', 'n_cells-0', 'feature_types-0-0-1', 'gene_symbol-0-0-1', 'n_cells-1-0-1', 'n_cells-1-1', 'n_cells'

In [70]:
adata.write(data_path + "Pilot1_Pilot2_filtered_ensembl_id_Jun29.h5ad")

... storing '10X_run' as categorical
... storing 'smartseq2_barcode' as categorical
... storing 'smartseq2_library_plate' as categorical
... storing 'smartseq2_sorting_direction' as categorical
... storing 'smartseq2_sort_order' as categorical
... storing 'smartseq2_cDNA_Plate' as categorical
... storing 'method' as categorical
... storing 'notes' as categorical
... storing 'donor' as categorical
... storing 'smartseq2_sorted_population' as categorical
... storing '10X_replicate' as categorical
... storing '10X_sample' as categorical
... storing 'sequencer' as categorical
... storing 'sex' as categorical
... storing 'anatomical_position' as categorical
... storing 'tissue' as categorical
... storing 'smartseq2_well_seq' as categorical
... storing '10X_barcode' as categorical
